In [1]:
import torch
import torch.nn as nn
import torchvision
import torchvision.transforms as tr

In [30]:
# Device configuration
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

#Hyper-parameters
num_epochs = 100
batch_size = 100
learning_rate = 0.001

# Image preprocessing models
transform = tr.Compose([
    tr.Pad(4),
    tr.RandomHorizontalFlip(),
    tr.RandomCrop(32),
    tr.ToTensor()])

# CIFAR-10 dataset
train_dataset = torchvision.datasets.CIFAR10(root='data/', train=True, transform=transform, download=True)

test_dataset = torchvision.datasets.CIFAR10(root='data/', train=False, transform=tr.ToTensor())

# Data Loader
train_loader = torch.utils.data.DataLoader(dataset=train_dataset, batch_size=batch_size, shuffle=True)

test_loader = torch.utils.data.DataLoader(dataset=test_dataset, batch_size=batch_size, shuffle=False)

Files already downloaded and verified


![resnetimage](https://user-images.githubusercontent.com/30661597/78585170-f4ac7c80-786b-11ea-8b00-8b751c65f5ca.PNG)

In [22]:
def conv3x3(in_channels, out_channels, stride=1):
    return nn.Conv2d(in_channels, out_channels, kernel_size=3, stride=stride, padding=1, bias=False)


class ResidualBlock(nn.Module):
    def __init__(self, in_channels, out_channels, stride=1, downsample=None):
        super(ResidualBlock, self).__init__()
        self.conv1 = conv3x3(in_channels, out_channels, stride)
        self.bn1 = nn.BatchNorm2d(out_channels)
        self.relu = nn.ReLU(inplace=True)
        self.conv2 = conv3x3(out_channels, out_channels)
        self.bn2 = nn.BatchNorm2d(out_channels)
        self.downsample = downsample

    def forward(self, x):
        residual = x.clone()
        out = self.conv1(x)
        out = self.bn1(out)
        out = self.relu(out)
        out = self.conv2(out)
        out = self.bn2(out)
        if self.downsample:
            residual = self.downsample(x)
        out += residual  # out= out + residual
        out = self.relu(out)
        return out

In [23]:
class ResNet(nn.Module):
    def __init__(self, block, layers, num_classes=10):
        """
        layers will be a list: [2,2,2]
        """
        super(ResNet, self).__init__()
        self.in_channels = 16
        self.conv = conv3x3(3, 16)
        self.bn = nn.BatchNorm2d(16)
        self.relu = nn.ReLU(inplace=True)
        self.layer1 = self.make_layer(block, 16, layers[0])
        self.layer2 = self.make_layer(block, 32, layers[1], 2)
        self.layer3 = self.make_layer(block, 64, layers[2], 2)
        self.avg_pool = nn.AvgPool2d(8)
        self.fc = nn.Linear(64, num_classes)

    def make_layer(self, block, out_channels, blocks, stride=1):
        downsample = None
        if (stride != 1) or (self.in_channels != out_channels):
            downsample = nn.Sequential(conv3x3(self.in_channels, out_channels, stride=stride),
                                       nn.BatchNorm2d(out_channels))
        layers = []
        layers.append(block(self.in_channels, out_channels, stride, downsample))
        self.in_channels = out_channels
        for i in range(1, blocks):
            layers.append(block(out_channels, out_channels))
        return nn.Sequential(*layers)

    def forward(self, x):
        out = self.conv(x)
        out = self.bn(out)
        out = self.relu(out)
        out = self.layer1(out)
        out = self.layer2(out)
        out = self.layer3(out)
        out = self.avg_pool(out)
        out = out.view(out.size(0), -1)
        out = self.fc(out)
        return out

In [32]:
model = ResNet(ResidualBlock, [2, 2, 2]).to(device)
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

In [33]:
optimizer.param_groups[0]['params']

[Parameter containing:
 tensor([[[[ 0.1620,  0.0228,  0.1398],
           [-0.0243,  0.1452,  0.0083],
           [-0.0067, -0.0022,  0.0309]],
 
          [[ 0.1725, -0.0709, -0.1744],
           [ 0.1012, -0.1385,  0.0228],
           [-0.0006, -0.1880, -0.1295]],
 
          [[-0.0034,  0.1325, -0.1155],
           [ 0.0216,  0.1336, -0.0044],
           [-0.0344, -0.0295,  0.0655]]],
 
 
         [[[ 0.1295,  0.1287,  0.0707],
           [-0.1076,  0.0946,  0.1167],
           [-0.0741,  0.1881,  0.1151]],
 
          [[ 0.1356, -0.0615, -0.1697],
           [-0.0958, -0.1554, -0.0446],
           [-0.1181, -0.1477,  0.1562]],
 
          [[ 0.1369,  0.1789, -0.0420],
           [ 0.1115, -0.1843, -0.0476],
           [ 0.0698,  0.1048, -0.1782]]],
 
 
         [[[-0.0315,  0.0974,  0.1536],
           [-0.0275, -0.1304,  0.1623],
           [-0.1417, -0.1663,  0.1347]],
 
          [[-0.0305,  0.1530, -0.1694],
           [-0.1170,  0.1388,  0.0998],
           [-0.1764,  0.1448, 

In [34]:
optimizer.param_groups[0]['lr']

0.001

In [35]:
decay = 0
model.train()
for epoch in range(num_epochs):

    # Decay the learning rate every 20 epochs
    if (epoch+1) % 20 == 0:
        decay+=1
        optimizer.param_groups[0]['lr'] = learning_rate * (0.5**decay)
        print("The new learning rate is {}".format(optimizer.param_groups[0]['lr']))

    for i, (images, labels) in enumerate(train_loader):
        images = images.to(device)
        labels = labels.to(device)
        outputs = model(images)
        loss = criterion(outputs, labels)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if (i+1) % 100 == 0:
            print ("Epoch [{}/{}], Step [{}/{}] Loss: {:.4f}"
                   .format(epoch+1, num_epochs, i+1, len(train_loader), loss.item()))

Epoch [1/100], Step [100/500] Loss: 1.7244
Epoch [1/100], Step [200/500] Loss: 1.6039
Epoch [1/100], Step [300/500] Loss: 1.3381
Epoch [1/100], Step [400/500] Loss: 1.0633
Epoch [1/100], Step [500/500] Loss: 0.9804
Epoch [2/100], Step [100/500] Loss: 1.0611
Epoch [2/100], Step [200/500] Loss: 1.0486
Epoch [2/100], Step [300/500] Loss: 1.0326
Epoch [2/100], Step [400/500] Loss: 1.0638
Epoch [2/100], Step [500/500] Loss: 0.9514
Epoch [3/100], Step [100/500] Loss: 0.9639
Epoch [3/100], Step [200/500] Loss: 0.9611
Epoch [3/100], Step [300/500] Loss: 0.9702
Epoch [3/100], Step [400/500] Loss: 0.9309
Epoch [3/100], Step [500/500] Loss: 0.9250
Epoch [4/100], Step [100/500] Loss: 0.7645
Epoch [4/100], Step [200/500] Loss: 0.8019
Epoch [4/100], Step [300/500] Loss: 0.6126
Epoch [4/100], Step [400/500] Loss: 0.7251
Epoch [4/100], Step [500/500] Loss: 0.6817
Epoch [5/100], Step [100/500] Loss: 0.7422
Epoch [5/100], Step [200/500] Loss: 0.6596
Epoch [5/100], Step [300/500] Loss: 0.9199
Epoch [5/10

In [36]:
# TEst the model
model.eval()
with torch.no_grad():
    correct = 0
    total = 0
    for images, labels in test_loader:
        images = images.to(device)
        labels = labels.to(device)
        outputs = model(images)
        _,predicted=torch.max(outputs.data,1)
        total += labels.size(0)
        correct += (predicted ==labels).sum().item()
    print('Accuracy of the model on the test images: {}%'.format(100*correct/total))

Accuracy of the model on the test images: 88.38%


In [38]:
torch.save(model.state_dict(),'../Trained_models/Rnn-SIFAR10 model/rnn_sifar_10')